In [ ]:
"""
Created on 20190316

float GA with DEAP

Ref: https://deap.readthedocs.io/en/master/tutorials/basic/part2.html
"""

In [1]:
import random
import numpy as np
from math import sin, cos, pi

from deap import base
from deap import creator
from deap import tools

# Implementation

In [32]:
IND_SIZE = 2
n_best = 1 # replace best indv as global best

creator.create("FitnessMax", base.Fitness, weights=(1.0, ))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate_gaft(individual):
    x, y = individual
    # constraint
    x = np.clip(x, -2, 2)
    y = np.clip(y, -2, 2)
    return y*sin(2*pi*x) + x*cos(2*pi*y),

toolbox.register("evaluate", evaluate_gaft)
# toolbox.register("mate", tools.cxBlend, alpha=0.2)
# toolbox.register("mate", tools.cxSimulatedBinaryBounded, eta=0.1, low=-2, up=2)
toolbox.register("mate", tools.cxSimulatedBinary, eta=0.1)
toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=5)

hof = tools.HallOfFame(1)

def main():
    # random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=300)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        
    hof.update(pop)
    
#     print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 100 and g < 1000:
        # A new generation
        g = g + 1
        if g % 100 == 0:
            print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # print("  Evaluated %i individuals" % len(invalid_ind))
        
        hof.update(offspring)
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        for i in range(n_best):
            # minimized (select the best several chromosomes)
            # pop[np.argpartition(fits, n_best)[:n_best][i]] = hof[i]
            # maximized
            pop[np.argpartition(fits, n_best)[:n_best][i]] = hof[i]
        
#         length = len(pop)
#         mean = sum(fits) / length
#         sum2 = sum(x*x for x in fits)
#         std = abs(sum2 / length - mean**2)**0.5
        
#         print("  Min %s" % min(fits))
#         print("  Max %s" % max(fits))
#         print("  Avg %s" % mean)
#         print("  Std %s" % std)
    
    # print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

In [33]:
main()

Start of evolution
-- Generation 100 --
-- Generation 200 --
-- Generation 300 --
-- Generation 400 --
-- Generation 500 --
-- Generation 600 --
-- Generation 700 --
-- Generation 800 --
-- Generation 900 --
-- Generation 1000 --
Best individual is [1.762678552778394, -7.01495493735823e+112], (3.7563359221353965,)


In [7]:
random.random

<function Random.random>

# Tutorial

In [2]:
IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

In [5]:
ind1 = toolbox.individual()
print(ind1)
print(ind1.fitness.valid)

[0.18568138267751166, 0.9718220086471654, 0.14470396007090336, 0.41193627476252903, 0.7932306559178652]
False


## Evaluation

In [6]:
def evaluate(individual):
    # Do some hard computing on the individual
    a = sum(individual)
    b = len(individual)
    return a, 1. / b

ind1.fitness.values = evaluate(ind1)
print(ind1.fitness.valid)    # True
print(ind1.fitness)          # (2.73, 0.2)

True
(2.5073742820759746, 0.2)


## Mutation

In [9]:
mutant = toolbox.clone(ind1)
ind2, = tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.2)
del mutant.fitness.values

print(ind2 is mutant)    # True
print(mutant is ind1)    # False

True
False


## Crossover

In [10]:
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxBlend(child1, child2, 0.5)
del child1.fitness.values
del child2.fitness.values
print(child1)
print(child2)

[0.32769061625591234, 0.9718220086471654, -0.2847363415594865, 0.41193627476252903, 0.7932306559178652]
[0.1643785275460457, 0.9718220086471654, 0.22381309343456698, 0.41193627476252903, 0.7932306559178652]


## Selection

In [13]:
selected = tools.selBest([child1, child2], 2)
print(child1 in selected)  # True

True


Usually duplication of the entire population will be made after selection or before variation.

In [ ]:
selected = toolbox.select(population, LAMBDA)
offspring = [toolbox.clone(ind) for ind in selected]

## Using the Toolbox

In [ ]:
from deap import base
from deap import tools

toolbox = base.Toolbox()

def evaluateInd(individual):
    # Do some computation
    return result,

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluateInd)

## Using the Tools

In [ ]:
for g in range(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = map(toolbox.clone, offspring)

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring

## Variations
`Variations allow to build simple algorithms using predefined small building blocks. In order to use a variation, the toolbox must be set to contain the required operators. For example in the lastly presented complete algorithm, the crossover and mutation are regrouped in the varAnd() function, this function requires the toolbox to contain the mate() and mutate() functions. This variation can be used to simplify the writing of an algorithm as follows.`

In [ ]:
from deap import algorithms

for g in range(NGEN):
    # Select and clone the next generation individuals
    offspring = map(toolbox.clone, toolbox.select(pop, len(pop)))

    # Apply crossover and mutation on the offspring
    offspring = algorithms.varAnd(offspring, toolbox, CXPB, MUTPB)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring

## Algorithms
`There are several algorithms implemented in the algorithms module. They are very simple and reflect the basic types of evolutionary algorithms present in the literature. The algorithms use a Toolbox as defined in the last sections. In order to setup a toolbox for an algorithm, you must register the desired operators under the specified names, refer to the documentation of the selected algorithm for more details. Once the toolbox is ready, it is time to launch the algorithm. The simple evolutionary algorithm takes 5 arguments, a population, a toolbox, a probability of mating each individual at each generation (cxpb), a probability of mutating each individual at each generation (mutpb) and a number of generations to accomplish (ngen).`

In [ ]:
from deap import algorithms

algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50)